#### The purpose of this notebook is to create  create a Machine Learning Classifier to identify True Vs Fake News

In [1]:
import pandas as pd
import numpy as np
# Importing datetime
from datetime import datetime
# Importing matplotlib
import matplotlib.pyplot as plt
# Importing re
import re
# Importing gensim, need to intall it before it can be used.
import gensim
# Importing spacy, need to install it before it can be used.
import spacy
# Run in terminal: python -m spacy download en
import textstat
from textblob import TextBlob
#Import statsmodel
import statsmodels.api as sm
#Import packages for data modeling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import en_core_web_sm
nlp = en_core_web_sm.load()
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from preprocess import Preprocesser
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Importing custom python modules
from ml_features import features
from ml_pipelines import pipelines

Using TensorFlow backend.


In [2]:
fake = pd.read_csv("data/Fake.csv")
fake["label"] = 0 #fake news label = 0
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [3]:
real = pd.read_csv("data/True.csv")
real["label"] = 1 # real news label = 1
real.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [4]:
news_df = pd.concat([fake, real])
news_df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [5]:
print(news_df.shape)
print("Number of labels of news dataset is ",news_df["label"].unique())

(44898, 5)
Number of labels of news dataset is  [0 1]


* Machine Learning Features are defined in ml_features.py module which consists of the feature engineering and splitting the news data into features and labels.

In [5]:
!pygmentize ml_features.py

import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time
import textstat
from textblob import TextBlob
#Import statsmodel
import statsmodels.api as sm
#Import packages for data modeling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

class features:
        
    def calculate_sentiment(self,title):
        """
        Function to calculate the sentiment polarity of news title
        """
        return TextBlob(title).sentiment.polarity


    def get_readability_score(self,text):
        """
        Function to calculate the readability score for the t

#### Creating Benchmark Model

In [6]:
Features = features()

In [42]:
X_train, X_test, y_train, y_test = Features.get_baseline_features(news_df)

In [43]:
benchmarkmodel = LogisticRegression()
benchmarkmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred = benchmarkmodel.predict(X_test)
print(classification_report(y_test,y_pred))
print("Test Accuracy of Benchmark Model is {}".format(accuracy_score(y_test,y_pred)))

              precision    recall  f1-score   support

           0       0.58      0.73      0.65      7037
           1       0.59      0.42      0.49      6433

    accuracy                           0.59     13470
   macro avg       0.59      0.58      0.57     13470
weighted avg       0.59      0.59      0.58     13470

Test Accuracy of Benchmark Model is 0.5864142538975501


The benchmark model used for classifying fake vs true news  has an Accuracy of 58% and Precision of 58% also

In [45]:
logit = sm.Logit(y_train, X_train)
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.682198
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  label   No. Observations:                31428
Model:                          Logit   Df Residuals:                    31425
Method:                           MLE   Df Model:                            2
Date:                Sun, 31 May 2020   Pseudo R-squ.:                 0.01426
Time:                        12:46:58   Log-Likelihood:                -21440.
converged:                       True   LL-Null:                       -21750.
Covariance Type:            nonrobust   LLR p-value:                1.944e-135
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
readability_score    -0.0062      0.000    -21.222      0.000      -0.007      -0.006
word_count         6.763e-05   2.66e-05      2.545      0.011    1.55e-05       0.000
sentiment_score       0.3140      0.043      7.253      0.000       0.229       0.399
=====================================================================================
"""

Based on the above results we see that readability score and sentiment score are correlated to the true news article because of significant coefficient values. The coefficient interpretations are not considered for interpretations because of less accuracy, precision and recall values. However, the benchmark model gives us an intuition about types of features to be considered in the model development.

#### Machine Learning Model Developement

In [7]:
# Extracting features to get features and labels using train and test split
X_train, X_test, y_train, y_test = Features.get_pipeline_features_labels(news_df)

Machine Learning pipelines are defined in ml_pipelines.py module which consists of the SVC and Naive-Bayes pipelines including the pre-processing steps in a sequential order. The advantage of modularizing the pipeliens was to sequentially apply the list of pre-processing transformations and classifiers.

In [8]:
!pygmentize ml_pipelines.py

import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time
import textstat
from textblob import TextBlob
import statsmodels.api as sm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from preprocess import Preprocesser

class pipelines:
    
    def create_svc_pipeline(self):
        """
        Function to create SVC classifier pipeline
        """
        pipeline = Pipeline([
                        ('vec', CountVectorizer(

In [8]:
Pipelines = pipelines()

In [14]:
# Create an SVC pipelines
svc_model = Pipelines.create_svc_pipeline()
svc_model.fit(X_train["text"],y_train)

Pipeline(memory=None,
         steps=[('vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=10,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='[a-zA-Z0-9]{3,}',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
           

In [15]:
y_pred = svc_model.predict(X_test["text"])
print(classification_report(y_test,y_pred))
print("Test Accuracy of SVC Classifier Model is {}".format(accuracy_score(y_test,y_pred)))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7037
           1       0.99      0.99      0.99      6433

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470

Test Accuracy of SVC Classifier Model is 0.9927988121752042


In [16]:
# Create a Multinomial-Naive Bayes pipeline
nvb_model = Pipelines.create_nvb_pipeline()
nvb_model.fit(X_train["text"],y_train)

Pipeline(memory=None,
         steps=[('vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=10,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='[a-zA-Z0-9]{3,}',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [17]:
y_pred = nvb_model.predict(X_test["text"])
print(classification_report(y_test,y_pred))
print("Test Accuracy of Multinomial-NaiveBayes Classifier Model is {}".format(accuracy_score(y_test,y_pred)))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      7037
           1       0.93      0.93      0.93      6433

    accuracy                           0.93     13470
   macro avg       0.93      0.93      0.93     13470
weighted avg       0.93      0.93      0.93     13470

Test Accuracy of Multinomial-NaiveBayes Classifier Model is 0.9319970304380104


#### Deep Learning Models

In [18]:
def create_keras_model(optimizer="adam", dropout=0.02):
    model = Sequential()
    model.add(Dense(50, activation='relu',input_dim = 1364381))
    model.add(Dropout(dropout), )
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=["accuracy"])
    return model

keras_clf = KerasClassifier(build_fn=create_keras_model, verbose=1, epochs=5)

keras_pipeline = Pipeline([("cv",CountVectorizer(min_df=0,max_df=1,ngram_range=(1,2))),
                            ("kc", keras_clf)])

In [19]:
keras_pipeline.fit(X_train["text"], y_train)

Epoch 1/5
31428/31428 [==============================] - 1108s 35ms/step - loss: 0.6921 - accuracy: 0.5222
Epoch 2/5
31428/31428 [==============================] - 1119s 36ms/step - loss: 0.1205 - accuracy: 0.9592
Epoch 3/5
31428/31428 [==============================] - 1144s 36ms/step - loss: 0.0694 - accuracy: 0.9798
Epoch 4/5
31428/31428 [==============================] - 1168s 37ms/step - loss: 0.0685 - accuracy: 0.9798
Epoch 5/5
31428/31428 [==============================] - 1174s 37ms/step - loss: 0.0678 - accuracy: 0.9799


Pipeline(memory=None,
         steps=[('cv',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1,
                                 max_features=None, min_df=0,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('kc',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x1a636cb990>)],
         verbose=False)

In [20]:
y_pred = keras_pipeline.predict(X_test["text"])

13470/13470 [==============================] - 123s 9ms/step


In [21]:
def bin_classifier(y_pred):
    for i in range(len(y_pred)):
        if(y_pred[i] >= 0.5):
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    return y_pred
    
y_pred = bin_classifier(y_pred)

In [23]:
print("Test Accuracy of Fully-Connected Feed Foward Model is {}".format(accuracy_score(y_pred,y_test)))

Test Accuracy of Fully-Connected Feed Foward Model is 0.8703043801039346


In [30]:
def create_keras_model(optimizer="adam", dropout=0.02):
    model = Sequential()
    model.add(Dense(20, activation='relu',input_dim = 1364381))
    model.add(Dropout(dropout), )
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=["accuracy"])
    return model

keras_clf = KerasClassifier(build_fn=create_keras_model, verbose=1, epochs=10)

keras_pipeline = Pipeline([("cv",CountVectorizer(min_df=0,max_df=1,ngram_range=(1,2))),
                           ('tfidf', TfidfTransformer()),
                            ("kc", keras_clf)])

In [31]:
keras_pipeline.fit(X_train["text"], y_train)

Epoch 1/10
31428/31428 [==============================] - 484s 15ms/step - loss: 0.6923 - accuracy: 0.5232
Epoch 2/10
31428/31428 [==============================] - 485s 15ms/step - loss: 0.1482 - accuracy: 0.9615
Epoch 3/10
31428/31428 [==============================] - 484s 15ms/step - loss: 0.0692 - accuracy: 0.9796
Epoch 4/10
31428/31428 [==============================] - 482s 15ms/step - loss: 0.0687 - accuracy: 0.9798
Epoch 5/10
31428/31428 [==============================] - 482s 15ms/step - loss: 0.0682 - accuracy: 0.9798
Epoch 6/10
31428/31428 [==============================] - 482s 15ms/step - loss: 0.0677 - accuracy: 0.9799
Epoch 7/10
31428/31428 [==============================] - 481s 15ms/step - loss: 0.0676 - accuracy: 0.9799
Epoch 8/10
31428/31428 [==============================] - 480s 15ms/step - loss: 0.0671 - accuracy: 0.9799
Epoch 9/10
31428/31428 [==============================] - 482s 15ms/step - loss: 0.0670 - accuracy: 0.9799
Epoch 10/10
31428/31428 [============

Pipeline(memory=None,
         steps=[('cv',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1,
                                 max_features=None, min_df=0,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('kc',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x1a5d3ba990>)],
         verbose=False)

In [32]:
y_pred = keras_pipeline.predict(X_test["text"])

13470/13470 [==============================] - 109s 8ms/step


In [33]:
def bin_classifier(y_pred):
    for i in range(len(y_pred)):
        if(y_pred[i] >= 0.5):
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    return y_pred
    
y_pred = bin_classifier(y_pred)

In [34]:
print("Test Accuracy of Fully-Connected Feed Foward Model is {}".format(accuracy_score(y_pred,y_test)))

Test Accuracy of Fully-Connected Feed Foward Model is 0.8838158871566444


#### Hyper-parameter Tuning

In [17]:
test_MultinomialNB = Pipeline([('vect', CountVectorizer(analyzer='word',
                                                       lowercase=True,
                                                        stop_words='english',
                                                       token_pattern='[a-zA-Z0-9]{3,}')),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB())])

In [18]:
from sklearn.model_selection import GridSearchCV

parameters = {'clf__alpha': np.arange(0.0,0.05,1.05)
}

grid_nb_clf = GridSearchCV(test_MultinomialNB,parameters,cv=5, n_jobs = 1)
_ = grid_nb_clf.fit(X_train["text"], y_train)

In [19]:
grid_nb_clf.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='[a-zA-Z0-9]{3,}',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=0.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [20]:
bestmultNB = grid_nb_clf.best_estimator_
bestmultNB.fit(X_train["text"],y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='[a-zA-Z0-9]{3,}',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=0.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [21]:
y_pred = bestmultNB.predict(X_test["text"])
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      7037
           1       0.96      0.91      0.93      6433

    accuracy                           0.94     13470
   macro avg       0.94      0.93      0.94     13470
weighted avg       0.94      0.94      0.94     13470



In [22]:
print("Test Accuracy of Multinomial-NaiveBayes Classifier Tuned Model is {}".format(accuracy_score(y_test,y_pred)))

Test Accuracy of Multinomial-NaiveBayes Classifier Tuned Model is 0.9358574610244988


In [9]:
test_svc = Pipeline([('vect', CountVectorizer(analyzer='word',
                                              lowercase=True,
                                              stop_words='english',
                                              token_pattern='[a-zA-Z0-9]{3,}')),
                 ('tfidf', TfidfTransformer()),
                 ('clf', LinearSVC())])

In [10]:
parameters = { 
              'clf__C':[1,10,100]
              }

grid_svc_clf = GridSearchCV(test_svc, parameters, cv=5, n_jobs = 1)
_ = grid_svc_clf.fit(X_train["text"], y_train)

In [11]:
grid_svc_clf.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='[a-zA-Z0-9]{3,}',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                   

In [13]:
bestlinearSVC = grid_svc_clf.best_estimator_
bestlinearSVC.fit(X_train["text"],y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='[a-zA-Z0-9]{3,}',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                   

In [14]:
y_pred = bestlinearSVC.predict(X_test["text"])
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7037
           1       0.99      0.99      0.99      6433

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [15]:
print("Test Accuracy of SVC Classifier Tuned Model is {}".format(accuracy_score(y_test,y_pred)))

Test Accuracy of SVC Classifier Tuned Model is 0.9928730512249443


#### Saving Models 

Models were saved in order to test a few sample news articles to understand how the data is performing on unseen news articles of current affairs.

In [31]:
from joblib import dump, load

In [33]:
dump(benchmarkmodel, './models/benchmarkmodel.joblib')
dump(svc_model, './models/svc_model.joblib')
dump(nvb_model, './models/nvb_model.joblib')

['./models/nvb_model.joblib']

In [32]:
svc_model = load('./models/svc_model.joblib') 

In [35]:
print(svc_model)

Pipeline(memory=None,
         steps=[('vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=10,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='[a-zA-Z0-9]{3,}',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
           

#### Sample News Testing

In [26]:
sample1 = ["Trump Signs Executive Order Directed at Social Media Companies After Feud With Twitter"]
y_pred = bestlinearSVC.predict(sample1)
# this should be real news - 1
print(y_pred)

[1]


### Results and Model Evaluation

| Model | Accuracy |
| --- | --- |
| Benchmark Logistic Regression | 58.64% |
| Multinomial Naive-Bayes | 93.12% |
| Linear SVC | 99% |
| Tuned Multinomial Naive-Bayes | 93.58% |
| Tuned Linear SVC | 99.28% |
| Fully Connected Feed-Forward Model (1-hidden layer – 50 nodes, dropout – 0.02, Adam optimizer) | 87.3% |
| Fully Connected Feed-Forward Model (1-hidden layer – 20 nodes, dropout – 0.02, Adam optimizer) | 88% |


#### Thus based on the above model results we chose the Tuned Linear SVC Model due to Out-of-Sample Accuracy of 99.28%